In [1]:
import nltk
import nltk.data
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
import spacy
import pandas as pd
import en_core_web_sm

C:\Users\User\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
nlp = en_core_web_sm.load()
stemmer = LancasterStemmer()

In [3]:
sentences = []

disc_sentences = {}

aux_list = ['am', 'are', 'is', 'was', 'were', 'can', 'could', 'does', 'do', 'did', 'has', 'had', 'may', 'might', 'must',
            'need', 'ought', 'shall', 'should', 'will', 'would']

discourse_markers = ['because', 'as a result', 'since', 'when', 'although', 'for example', 'for instance']

qtype = {'because': ['Why'], 'since': ['When', 'Why'], 'when': ['When'], 'although': ['Yes/No'], 'as a result': ['Why'], 
        'for example': ['Give an example where'], 'for instance': ['Give an instance where'], 'to': ['Why']}

target_arg = {'because': 0, 'since': 0, 'when': 0, 'although': 0, 'as a result': 1, 'for example': 0, 'for instance': 0, 
              'to': 0}

In [4]:
# tokenize and split into sentences
def sentensify():
    global sentences
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    fp = open('Harry Potter.txt')
    data = fp.read()
    sentences = tokenizer.tokenize("I will clean the house every Saturday.")
#     sentences = tokenizer.tokenize("My name is Abhishek.")
#     sentences = tokenizer.tokenize(data)
    discourse()

In [5]:
def get_questions(question_part, type):
          
    for i in range(0, len(question_part)):
        if(question_part[i] == 'i'):
            if((i == 0 and question_part[i+1] == ' ') or (question_part[i-1] == ' ' and question_part[i+1] == ' ')):
                question_part = question_part[:i] + 'I' + question_part[i + 1: ]
                
    question = ""
    if(type == 'Give an example where' or type == 'Give an instance where'):
        question = type + " " + question_part + '?'
        return question

    aux_verb = False
    res = None
    
    # check if auxiliary verb exists
    for i in range(len(aux_list)):
        if(aux_list[i] in question_part.split()):
            aux_verb = True
            pos = i
            break

    # If auxiliary verb exists
    if(aux_verb):
#         print("yes")
        text = nltk.word_tokenize(question_part)
        tags = nltk.pos_tag(text)
        question_part = ""
        fP = False
        
        for word, tag in tags:
            if(word in ['I', 'We', 'we']):
                question_part += 'you' + " "
                fP = True
                continue
            question_part += word + " "

        question = question_part.split(" " + aux_list[pos])
        if(fP):
             question = ["were "] + question
        else:
            question = [aux_list[pos] + " "] + question

        # If Yes/No, no need to introduce question phrase
        if(type == 'Yes/No'):
            question += ['?']
            
        elif(type != "non_disc"):
            question = [type + " "] + question + ["?"]
            
        else:
            question = question + ["?"]
         
        question = ''.join(question)

    
    else:
#         print("no")
        aux = None
        text = nltk.word_tokenize(question_part)
        tags = nltk.pos_tag(text)
        comb = ""

        '''There can be following combinations of nouns and verbs:
            NN/NNP and VBZ  -> Does
            NNS/NNPS(plural) and VBP -> Do
            NN/NNP and VBN -> Did
            NNS/NNPS(plural) and VBN -> Did
        '''
        
        for tag in tags:
            if(comb == ""):
                if(tag[1] == 'NN' or tag[1] == 'NNP'):
                    comb = 'NN'

                elif(tag[1] == 'NNS' or tag[1] == 'NNPS'):
                    comb = 'NNS'

                elif(tag[1] == 'PRP'):
                    if tag[0] in ['He','She','It']:
                        comb = 'PRPS'
                    else:
                        comb = 'PRPP'
                        tmp = question_part.split(" ")
                        tmp = tmp[1: ]
                        if(tag[0] in ['I', 'we', 'We']):
                            question_part = 'you ' + ' '.join(tmp)
                            
            if(res == None):
                res = re.match(r"VB*", tag[1])
                if(res):
                    
                    # Stem the verb
                    question_part = question_part.replace(tag[0], stemmer.stem(tag[0]))
                res = re.match(r"VBN", tag[1])
                res = re.match(r"VBD", tag[1])

        if(comb == 'NN'):
            aux = 'does'
            
        elif(comb == 'NNS'):
            aux = 'do'
            
        elif(comb == 'PRPS'):
            aux = 'does'
            
        elif(comb == 'PRPP'):
            aux = 'do'
            
        if(res in ['VBD', 'VBN']):
            aux = 'did'

        if(aux):
            if(type == "non_disc" or type == "Yes/No"):
                question = aux + " " + question_part + "?"

            else:
                question = type + " " + aux + " " + question_part + "?"
    if(question != ""):
        question = question[0].upper() + question[1:]
    return question

In [6]:
def discourse():
    temp = []
    target = ""
    questions = []
    global disc_sent
    disc_sent = {}
    for i in range(len(sentences)):
#         maxLen = 9999999
        val = -1
        for j in discourse_markers:
            tmp = len(sentences[i].split(j)[0].split(' '))
            if(len(sentences[i].split(j)) > 1):
#                 maxLen = tmp
                val = j
        if(val != -1):

            # To initialize a list for every new key
            if(disc_sent.get(val, 'empty') == 'empty'):
                disc_sent[val] = []
            
            disc_sent[val].append(sentences[i])
            temp.append(sentences[i])
    
    nondisc_sent = list(set(sentences) - set(temp))
#     print(disc_sent)
    t = []
    for k, v in disc_sent.items():
        for val in range(len(v)):
            
            # Split the sentence on discourse marker and identify the question part
            question_part = disc_sent[k][val].split(k)[target_arg[k]]
#             print(question_part)
            q = get_questions(question_part, qtype[k][0])
            if(q != ""):
                questions.append([disc_sent[k][val],q])
                
    nondisc_sent = list(set(sentences) - set(temp))
#     print(nondisc_sent)
                
                
    for question_part in nondisc_sent:
        s = "non_disc"
        sentence = question_part
        text = nltk.word_tokenize(question_part)
        if(text[0] == 'Yes'):
            question_part = question_part[5:]
            s = "Yes/No"
            
        elif(text[0] == 'No'):
            question_part = question_part[4:]
            s = "Yes/No"
            
        q = get_questions(question_part, s)
        if(q != ""):
            questions.append([sentence,q])
    print(len(questions))
    
    for pair in questions:
        print("S: ",pair[0])
        print("Q: ",pair[1])
        print()

In [7]:
sentensify()

1
S:  I will clean the house every Saturday.
Q:  Were you clean the house every Saturday . ?

